In [1]:
import numpy as np
import pandas as pd

In [32]:
df_itm_cat = pd.read_csv('00-Input/item_categories.csv')
df_itm = pd.read_csv('00-Input/items.csv')
df_sh = pd.read_csv('00-Input/shops.csv')
df_tr = pd.read_csv('00-Input/sales_train.csv')
df_te = pd.read_csv('00-Input/test.csv')
df_sub = pd.read_csv('00-Input/sample_submission.csv')

In [3]:
# Cleaning shops data

df_tr.loc[df_tr.shop_id == 0, 'shop_id'] = 57
df_te.loc[df_te.shop_id == 0, 'shop_id'] = 57

df_tr.loc[df_tr.shop_id == 1, 'shop_id'] = 58
df_te.loc[df_te.shop_id == 1, 'shop_id'] = 58

df_tr.loc[df_tr.shop_id == 10, 'shop_id'] = 11
df_te.loc[df_te.shop_id == 10, 'shop_id'] = 11

In [4]:
# Filter pandas DataFrame by substring critera
df_sh[df_sh['shop_name'].str.contains('! ', regex=False)]

,city,shop_name,shop_id
0,Yakutsk,"! Yakutsk Ordzhonikidze, 56 Franc",0
1,Yakutsk,"! Yakutsk TC ""Central"" Franc",1


In [5]:
# getting rid of "!" before shop_names
df_sh['shop_name'] = df_sh['shop_name'].map(lambda x: x.split('!')[1] if x.startswith('!') else x)

In [6]:
df_tr['date'] = pd.to_datetime(df_tr['date'],format = '%d.%m.%Y')
df_tr

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [7]:
# making a dataset with only monthly sales data

data = df_tr.groupby(['date_block_num', 'shop_id', 'item_id']).sum().reset_index()
data

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,2,27,2499.0,1.0
1,0,2,33,499.0,1.0
2,0,2,317,299.0,1.0
3,0,2,438,299.0,1.0
4,0,2,471,798.0,2.0
...,...,...,...,...,...
1609119,33,59,22087,357.0,6.0
1609120,33,59,22088,238.0,2.0
1609121,33,59,22091,179.0,1.0
1609122,33,59,22100,629.0,1.0


In [8]:
# now we will create a pivot tabel by going so we get our data in desired form 
# we want get total count value of an item over the whole month for a shop 
# That why we made shop_id and item_id our indices and date_block_num our column 
# the value we want is item_cnt_day and used sum as aggregating function

data = df_tr.pivot_table(index=['shop_id','item_id'], values = 'item_cnt_day', columns = 'date_block_num', fill_value = 0, aggfunc='sum').reset_index()
data

date_block_num,shop_id,item_id,0,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,32,33
0,2,27,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,30,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,31,0,4,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,32,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,2,33,1,0,0,0,0,0,0,0,...,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418904,59,22154,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
418905,59,22155,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
418906,59,22162,0,0,0,0,0,0,0,0,...,0,9,4,1,1,0,0,1,0,0
418907,59,22164,0,0,0,0,0,0,0,0,...,0,2,1,2,0,0,1,0,0,0


In [9]:
# Now we will merge our pivot table with the test_data because we want to keep the data of items we have
# predict

dataset = pd.merge(df_te,data,on = ['item_id','shop_id'],how = 'left')
dataset

,ID,shop_id,item_id,0,1,2,3,4,5,6,...,24,25,26,27,28,29,30,31,32,33
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# lets fill all NaN values with 0
dataset.fillna(0,inplace = True)
# lets check our data now 
dataset.head()

,ID,shop_id,item_id,0,1,2,3,4,5,6,...,24,25,26,27,28,29,30,31,32,33
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# we will drop shop_id and item_id because we do not need them
# we are teaching our model how to generate the next sequence

#dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
#dataset.head()

In [15]:
# now let's create the actual training data

X = dataset.drop([33, 'ID', 'item_id', 'shop_id'], axis = 1)
y = dataset[33]

# deleting the first column so that it can predict the future sales data
x_test = dataset.drop([0, 'ID', 'item_id', 'shop_id'], axis = 1)

# checking the shapes of the datasets
print("Shape of x_train :", X.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_test :", y.shape)

Shape of x_train : (214200, 33)
Shape of x_test : (214200, 33)
Shape of y_test : (214200,)


In [16]:
# splitting the data into train and valid dataset

from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.1, random_state = 0)

# checking the shapes
print("Shape of x_train :", x_train.shape)
print("Shape of x_valid :", x_valid.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of y_valid :", y_valid.shape)

Shape of x_train : (192780, 33)
Shape of x_valid : (21420, 33)
Shape of y_train : (192780,)
Shape of y_valid : (21420,)


## LSTM

In [12]:
# importing libraries required for our model
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [13]:
# defining model 
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [17]:
history= my_model.fit(x_train,y_train,batch_size = 1000,epochs = 20, validation_data=(x_valid, y_valid))

Epoch 1/20
193/193 [==============================] - 66s 340ms/step - loss: 5.6134 - mean_squared_error: 5.6134 - val_loss: 228.8991 - val_mean_squared_error: 228.8991
Epoch 2/20
193/193 [==============================] - 62s 321ms/step - loss: 5.4912 - mean_squared_error: 5.4912 - val_loss: 228.8202 - val_mean_squared_error: 228.8202
Epoch 3/20
193/193 [==============================] - 50s 257ms/step - loss: 5.6283 - mean_squared_error: 5.6283 - val_loss: 228.7475 - val_mean_squared_error: 228.7475
Epoch 4/20
193/193 [==============================] - 50s 259ms/step - loss: 5.5288 - mean_squared_error: 5.5288 - val_loss: 228.6082 - val_mean_squared_error: 228.6082
Epoch 5/20
193/193 [==============================] - 53s 272ms/step - loss: 5.5327 - mean_squared_error: 5.5327 - val_loss: 228.5733 - val_mean_squared_error: 228.5733
Epoch 6/20
193/193 [==============================] - 54s 279ms/step - loss: 5.4127 - mean_squared_error: 5.4127 - val_loss: 228.5452 - val_mean_squared_er

In [18]:
from sklearn.metrics import mean_squared_error

y_pred = my_model.predict(x_valid)
print('LSTM RMSE =', mean_squared_error(y_valid, y_pred, squared=False))

LSTM RMSE = 15.082711510558164


## XGB

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing


In [20]:
from xgboost import XGBRegressor

RANDOM_STATE = 42

xgb_model = XGBRegressor(
    # learning_rate=0.05,
    max_depth=16,
    n_estimators=200,
    seed=RANDOM_STATE,
)

xgb_model.fit(x_train, y_train, early_stopping_rounds=10, eval_metric="rmse",
              eval_set=[(x_train, y_train), (x_valid, y_valid)], verbose=10)

[0]	validation_0-rmse:2.22552	validation_1-rmse:15.31165
[10]	validation_0-rmse:1.18412	validation_1-rmse:14.93344
[20]	validation_0-rmse:1.16661	validation_1-rmse:14.90837
[30]	validation_0-rmse:1.16378	validation_1-rmse:14.90751
[40]	validation_0-rmse:1.15717	validation_1-rmse:14.90729
[44]	validation_0-rmse:1.15585	validation_1-rmse:14.90735


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=16, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [21]:
y_pred = xgb_model.predict(x_valid)
print('XGBoost RMSE =', mean_squared_error(y_valid, y_pred, squared=False))

XGBoost RMSE = 14.907301502200601


In [23]:
from lightgbm import LGBMRegressor

lgbm_model = LGBMRegressor(
#     learning_rate=0.05,
    max_depth=16,
    n_estimators=200,
    seed=RANDOM_STATE,
)

lgbm_model.fit(x_train, y_train, early_stopping_rounds=10, eval_metric="rmse",
               eval_set=[(x_train, y_train), (x_valid, y_valid)], verbose=10)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[10]	training's rmse: 2.10821	training's l2: 4.44457	valid_1's rmse: 14.7134	valid_1's l2: 216.485
[20]	training's rmse: 1.94157	training's l2: 3.76969	valid_1's rmse: 14.4678	valid_1's l2: 209.316
[30]	training's rmse: 1.84548	training's l2: 3.40579	valid_1's rmse: 14.4175	valid_1's l2: 207.863
[40]	training's rmse: 1.78044	training's l2: 3.16998	valid_1's rmse: 14.3672	valid_1's l2: 206.417
[50]	training's rmse: 1.73234	training's l2: 3.00099	valid_1's rmse: 14.3359	valid_1's l2: 205.519
[60]	training's rmse: 1.69208	training's l2: 2.86312	valid_1's rmse: 14.2934	valid_1's l2: 204.303
[70]	training's rmse: 1.65908	training's l2: 2.75253	valid_1's rmse: 14.2829	valid_1's l2: 204.001
[80]	training's rmse: 1.6246	training's l2: 2.63933	valid_1's rmse: 14.2727	valid_1's l2: 203.709
[90]	training's rmse: 1.59697	training's l2: 2.55032	valid_1's rmse: 14.2701	valid_1's l2: 203.635


LGBMRegressor(max_depth=16, n_estimators=200, seed=42)

In [24]:
y_pred = lgbm_model.predict(x_valid)
print('LGBM RMSE =', mean_squared_error(y_valid, y_pred, squared=False))

LGBM RMSE = 14.25934053977522


In [34]:
df_result = df_sub.copy()
df_result['XGB'] = xgb_model.predict(x_test)
df_result['LGBM'] = lgbm_model.predict(x_test)
df_result['LSTM'] = my_model.predict(x_test)
df_result

,ID,item_cnt_month,XGB,LGBM,LSTM
0,0,0.5,0.637927,0.483158,0.488356
1,1,0.5,0.129416,0.126086,0.133795
2,2,0.5,0.468254,0.780542,0.799321
3,3,0.5,0.145604,0.167402,0.153716
4,4,0.5,0.129416,0.126086,0.133795
...,...,...,...,...,...
214195,214195,0.5,0.294121,0.288173,0.421201
214196,214196,0.5,0.129416,0.126086,0.133795
214197,214197,0.5,0.047644,0.126086,0.108702
214198,214198,0.5,0.129416,0.126086,0.133795


In [ ]:
xgb_model.plot_importance(model)
plt.show()